In [61]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [62]:
import numpy as np
import pandas as pd

In [8]:
df = pd.read_csv("spam.csv",encoding = "ISO-8859-1")
df.drop(columns= df.columns[-3:], inplace=True)

In [9]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
df.v1.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [11]:
df_ham,df_spam = df.groupby(df.v1)

In [12]:
df_ham=df_ham[1]

In [13]:
df_ham

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...
...,...,...
5565,ham,Huh y lei...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [14]:
df_spam = df_spam[1]

In [15]:
df_spam

,v1,v2
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [16]:
df_spam_resample = df_spam.sample(df_ham.shape[0],replace=True)

In [17]:
df_spam_resample

,v1,v2
3420,spam,Welcome! Please reply with your AGE and GENDER...
838,spam,We tried to contact you re our offer of New Vi...
4585,spam,"Mila, age23, blonde, new in UK. I look sex wit..."
2123,spam,+123 Congratulations - in this week's competit...
3895,spam,tells u 2 call 09066358152 to claim å£5000 pri...
...,...,...
1104,spam,Message Important information for O2 user. Tod...
823,spam,25p 4 alfie Moon's Children in need song on ur...
4582,spam,For ur chance to win a å£250 wkly shopping spr...
1448,spam,As a registered optin subscriber ur draw 4 å£1...


In [18]:
df_new=pd.concat([df_ham,df_spam_resample])

In [19]:
df_new=df_new.sample(frac=1).reset_index(drop=True)

In [20]:
df_new.head()

,v1,v2
0,spam,Get ur 1st RINGTONE FREE NOW! Reply to this ms...
1,spam,New TEXTBUDDY Chat 2 horny guys in ur area 4 j...
2,spam,You are a winner you have been specially selec...
3,spam,You have WON a guaranteed å£1000 cash or a å£2...
4,spam,Talk sexy!! Make new friends or fall in love i...


In [21]:
df_new.v1.value_counts()

spam    4825
ham     4825
Name: v1, dtype: int64

In [22]:
df_new['spam']=pd.get_dummies(df_new.v1,drop_first=True)

In [51]:
df_new.head(10)

,v1,v2,spam
0,spam,Get ur 1st RINGTONE FREE NOW! Reply to this ms...,1
1,spam,New TEXTBUDDY Chat 2 horny guys in ur area 4 j...,1
2,spam,You are a winner you have been specially selec...,1
3,spam,You have WON a guaranteed å£1000 cash or a å£2...,1
4,spam,Talk sexy!! Make new friends or fall in love i...,1
5,ham,I hope you arnt pissed off but id would really...,0
6,spam,You have WON a guaranteed å£1000 cash or a å£2...,1
7,ham,make that 3! 4 fucks sake?! x,0
8,spam,I am hot n horny and willing I live local to y...,1
9,spam,"accordingly. I repeat, just text the word ok o...",1


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_new['v2'],df_new['spam'],test_size=0.2)

In [66]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [60]:
# For fine-tune bert model
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable = True)

In [67]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [68]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
242/242 [==============================] - 262s 1s/step - loss: 0.4287 - accuracy: 0.8347
Epoch 2/10
242/242 [==============================] - 301s 1s/step - loss: 0.2707 - accuracy: 0.9122
Epoch 3/10
242/242 [==============================] - 316s 1s/step - loss: 0.2221 - accuracy: 0.9313
Epoch 4/10
242/242 [==============================] - 301s 1s/step - loss: 0.1908 - accuracy: 0.9400
Epoch 5/10
242/242 [==============================] - 293s 1s/step - loss: 0.1724 - accuracy: 0.9473
Epoch 6/10
242/242 [==============================] - 289s 1s/step - loss: 0.1614 - accuracy: 0.9509
Epoch 7/10
242/242 [==============================] - 288s 1s/step - loss: 0.1501 - accuracy: 0.9532
Epoch 8/10
242/242 [==============================] - 293s 1s/step - loss: 0.1418 - accuracy: 0.9587
Epoch 9/10
242/242 [==============================] - 294s 1s/step - loss: 0.1356 - accuracy: 0.9609
Epoch 10/10
242/242 [==============================] - 301s 1s/step - loss: 0.1274 - accura

In [30]:
model.save('bert_classifier.model')

INFO:tensorflow:Assets written to: bert_classifier.model\assets


INFO:tensorflow:Assets written to: bert_classifier.model\assets


In [31]:
model.evaluate(X_test, y_test)

61/61 [==============================] - 58s 931ms/step - loss: 0.1174 - accuracy: 0.9642


[0.1173933818936348, 0.9642487168312073]

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

In [35]:
y_predicted = model.predict(X_test)

In [39]:
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)

In [40]:
confusion_matrix(y_test, y_predicted)

array([[946,  24],
       [ 45, 915]], dtype=int64)